Según las webadas que leo, el csp responde a esta simple ecuación:

$$S = WA$$

Donde W es el resultado del algoritmo CSP (que al parecer proviene de la rama de procesamiento de señales). $A$ es la matriz de entrada (que al parecer tiene más dimensiones de las que pensaba - channels $\times$ samples), y S es la matriz de salida (que debería ser sources $\times$ samples).

Por eso voy a analizar linea por línea el código de la [función fit de mne](https://github.com/mne-tools/mne-python/blob/maint/0.19/mne/decoding/csp.py#L144-L255).

## Definiendo variables

$X$ e $y$ son los inputs. $X$ es la matriz de epochs, donde cada epoch es una matriz de channels $\times$ samples (epochs $\times$ channels $\times$ samples). $y$, por otro lado, es un arreglo con las clases de cada epoch.

Otros datos que se necesitan son:
* Número de canales (que se obtiene de $X$)
* Número de classes (que se obtiene de $y$)
* Una matriz de covarianza (que es classes $\times$ channels $\times$ channels y no sé por qué)
* Una lista con los pesos de las muestras o sample (supongo que cada epoch debe tener un número igual de samples para que funcione)

In [ ]:
self._classes = np.unique(y)

n_channels = X.shape[1]
n_classes = len(self._classes)

covs = np.zeros((n_classes, n_channels, n_channels))
sample_weights = list()

# Calculando la matriz de covarianza por clase

Supongo que la matriz de convarianza es de channels $\times$ channels, o 14 $\times$ 14.

In [ ]:
for class_idx, this_class in enumerate(self._classes):
    class_ = np.transpose(X[y == this_class], [1, 0, 2])
    class_ = class_.reshape(n_channels, -1)
    cov = _regularized_covariance(class_, reg=self.reg, method_params=self.cov_method_params, rank=self.rank)
    weight = sum(y == this_class)

    covs[class_idx] = cov

Vamos a detenernos en esta línea. Se seleccionan solo los epochs pertenecientes a esa clase con `X[y == this_class]`. Y luego se transpone de tal forma que, antes:

* Teníamos los epochs, y dentro de cada epoch teníamos todos los canales

Y ahora:

* Tenemos todos los canales, y dentro de cada uno los epochs

Si la finalidad es convertir una matriz de channels $\times$ samples a una de sources $\times$ samples, yo diría que es el camino correcto... creo.

In [ ]:
class_ = np.transpose(X[y == this_class], [1, 0, 2])

In [9]:
import numpy as np

# epochs × channels × samples
epochs = 5 # 5 epochs of 1 seg each
channels = 5 # Insight
samples = 10 # 10Hz
X = np.zeros((epochs, channels, samples))

for i in range(0, len(X)):
    X[i][0][0] = i + 1
    
X

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [13]:
y = np.random.randint(0, 2, size=epochs)
y

array([1, 1, 1, 0, 0])

In [18]:
X[y == 1] # epochs x channels x samples

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [25]:
class_ = np.transpose(X[y == 1], [1, 0, 2]) # channels x epochs x samples
class_

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [3., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

Aquí estoy concatenando los epochs por channel

In [32]:
class_ = class_.reshape(channels, -1)
class_

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

Hasta aquí llegué, necesito revisar las cosas de algebra lineal

In [33]:
cov = _regularized_covariance(class_, reg=self.reg, method_params=self.cov_method_params, rank=self.rank)

NameError: name '_regularized_covariance' is not defined

In [ ]:
def _regularized_covariance(data, reg=None, method_params=None, info=None,
                            rank=None):
    """Compute a regularized covariance from data using sklearn.
    This is a convenience wrapper for mne.decoding functions, which
    adopted a slightly different covariance API.
    Returns
    -------
    cov : ndarray, shape (n_channels, n_channels)
        The covariance matrix.
    """
    if reg is None:
        reg = 'empirical'
    try:
        reg = float(reg)
    except ValueError:
        pass
    if isinstance(reg, float):
        if method_params is not None:
            raise ValueError('If reg is a float, method_params must be None '
                             '(got %s)' % (type(method_params),))
        method_params = dict(shrinkage=dict(
            shrinkage=reg, assume_centered=True, store_precision=False))
        reg = 'shrinkage'
    elif not isinstance(reg, str):
        raise ValueError('reg must be a float, str, or None, got %s (%s)'
                         % (reg, type(reg)))
    method, method_params = _check_method_params(
        reg, method_params, name='reg', allow_auto=False, rank=rank)
    # use mag instead of eeg here to avoid the cov EEG projection warning
    info = create_info(data.shape[-2], 1000., 'mag') if info is None else info
    picks_list = _picks_by_type(info)
    scalings = _handle_default('scalings_cov_rank', None)
    cov = _compute_covariance_auto(
        data.T, method=method, method_params=method_params,
        info=info, cv=None, n_jobs=1, stop_early=True,
        picks_list=picks_list, scalings=scalings,
        rank=rank)[reg]['data']
    return cov